In [79]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from bs4 import BeautifulSoup as BS
from lxml import html
import re
import difflib
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
imdbdf = pd.read_csv("../data/imdbdf.csv")

In [45]:
imdbdf = imdbdf.drop("Unnamed: 0",axis=1)

In [3]:
imdbdf.loc[(imdbdf['Title'] == 'SMILE')]

Unnamed: 0  Title  Release Year     Rating  Runtime IMDB Score     Metascore
3              4  SMILE        2022.0          R    115.0        6.6            68
1006        1007  SMILE        2009.0  Not Rated     84.0        3.3  NO METASCORE
1698        1699  SMILE        2022.0  NO RATING      NaN        3.8  NO METASCORE

In [4]:
imdbdf = imdbdf.drop(1698)

In [90]:
imdbdf.loc[(imdbdf['Title'] == 'SMILE')]

Title Release Year     Rating Runtime IMDB Score     Metascore Title and Year
4    SMILE       2022.0          R   115.0        6.6            68   SMILE 2022.0
999  SMILE       2009.0  Not Rated    84.0        3.3  NO METASCORE   SMILE 2009.0

In [6]:
imdbdf['Release Year'] = pd.to_numeric(imdbdf['Release Year'])
imdbdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    9999 non-null   int64  
 1   Title         9999 non-null   object 
 2   Release Year  9607 non-null   float64
 3   Rating        9999 non-null   object 
 4   Runtime       9234 non-null   float64
 5   IMDB Score    9999 non-null   object 
 6   Metascore     9999 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 624.9+ KB


In [7]:
imdbdf = imdbdf[imdbdf['Release Year'].notna()]

In [8]:
imdbdf.index = np.arange(1, len(imdbdf) + 1)
imdbdf['Title'] = imdbdf['Title'].str.upper()

In [80]:
imdbdf['Title'] = imdbdf['Title'].fillna(value=1408)

In [91]:
imdbdf.head(300)

Title Release Year     Rating Runtime      IMDB Score     Metascore                                     Title and Year
1                                             THE MENU       2022.0          R   107.0             7.5            71                                    THE MENU 2022.0
2                                                    X       2022.0          R   105.0             6.6            79                                           X 2022.0
3                                        BONES AND ALL       2022.0          R   131.0             7.3            74                               BONES AND ALL 2022.0
4                                                SMILE       2022.0          R   115.0             6.6            68                                       SMILE 2022.0
5                                            BARBARIAN       2022.0          R   102.0             7.1            78                                   BARBARIAN 2022.0
6                                              BHEDIYA       2022.0  NO RATING   156.0             7.7  NO METASCORE                                     BHEDIYA 2022.0
7                                                 NOPE       2022.0          R   130.0             6.9            77                                        NOPE 2022.0
8                                         THE MEAN ONE       2022.0  NO RATING    93.0             6.3            32                                THE MEAN ONE 2022.0
9                                          TERRIFIER 2       2022.0  Not Rated   138.0             6.3            59                                 TERRIFIER 2 2022.0
10                                               PEARL       2022.0          R   103.0             7.0            73                                       PEARL 2022.0
11                                              SCREAM       2022.0          R   114.0             6.3            60                                      SCREAM 2022.0
12                                         WHITE NOISE       2022.0          R   136.0             6.6            67                                 WHITE NOISE 2022.0
13                               SOMETHING IN THE DIRT       2022.0          R   116.0             6.0            76                       SOMETHING IN THE DIRT 2022.0
14                        THE BABYSITTER: KILLER QUEEN       2020.0      TV-MA   101.0             5.8            22                THE BABYSITTER: KILLER QUEEN 2020.0
15                                         SKINAMARINK       2022.0  NO RATING   100.0             5.5  NO METASCORE                                 SKINAMARINK 2022.0
16                                   THE ADDAMS FAMILY       2019.0         PG    86.0             5.8            46                           THE ADDAMS FAMILY 2019.0
17                                           TERRIFIER       2016.0    Unrated    85.0             5.6  NO METASCORE                                   TERRIFIER 2016.0
18                    WINNIE-THE-POOH: BLOOD AND HONEY       2023.0  NO RATING     nan  NO IMDB RATING  NO METASCORE            WINNIE-THE-POOH: BLOOD AND HONEY 2023.0
19                                           MIDSOMMAR       2019.0          R   148.0             7.1            72                                   MIDSOMMAR 2019.0
20                                            SCREAM 6       2023.0  NO RATING     nan  NO IMDB RATING  NO METASCORE                                    SCREAM 6 2023.0
21                                     AMERICAN PSYCHO       2000.0          R   102.0             7.6            64                             AMERICAN PSYCHO 2000.0
22                                BODIES BODIES BODIES       2022.0          R    94.0             6.3            69                        BODIES BODIES BODIES 2022.0
23                                             GET OUT       2017.0          R   104.0             7.7            85                                     GET OUT 2017.0
24         DOCTOR

In [9]:
wwbo = pd.read_excel("../data/thenumbers.xlsx", sheet_name=('All Time Worldwide Box Office'))
wwbo.index = np.arange(1, len(wwbo) + 1)
wwbo = wwbo.rename(columns={'Movie' : 'Title', 'Released' : 'Release Year'})
wwbo['Worldwide']=wwbo['Worldwide'].apply('{:,}'.format)
wwbo['Domestic']=wwbo['Domestic'].apply('{:,}'.format)
wwbo['International']=wwbo['International'].apply('{:,}'.format)

In [10]:
wwbo['Title'] = wwbo['Title'].str.upper()

In [84]:
wwbo

Release Year                                 Title    Worldwide       Domestic  International                             Title and Year
1            2017                                    IT  701,012,746  328,828,874.0  372,183,872.0                                    IT 2017
2            1975                                  JAWS  482,947,378  272,257,035.0  210,690,343.0                                  JAWS 1975
3            2007                           I AM LEGEND  585,532,684  256,393,010.0  329,139,674.0                           I AM LEGEND 2007
4            1973                          THE EXORCIST  428,214,478  230,347,346.0  197,867,132.0                          THE EXORCIST 1973
5            2019                       IT: CHAPTER TWO  467,563,955  211,593,228.0  255,970,727.0                       IT: CHAPTER TWO 2019
...           ...                                   ...          ...            ...            ...                                        ...
1851         2018                          MANIAC TALES          155            nan          155.0                          MANIAC TALES 2018
1852         1933                     THE INVISIBLE MAN          132            nan          132.0                     THE INVISIBLE MAN 1933
1853         2016                  CAPTURE KILL RELEASE           72            nan           72.0                  CAPTURE KILL RELEASE 2016
1854         2021                SLUMBER PARTY MASSACRE           28            nan           28.0                SLUMBER PARTY MASSACRE 2021
1855         2019  ANTRUM: THE DEADLIEST FILM EVER MADE           26            nan           26.0  ANTRUM: THE DEADLIEST FILM EVER MADE 2019

[1855 rows x 6 columns]

In [85]:
#df[['a', 'b']] = df[['a','b']].fillna(value=0)

wwbo['Title'] = wwbo['Title'].fillna(value=1408)

In [86]:
wwbo.head(100)

Release Year                                     Title    Worldwide       Domestic  International                                 Title and Year
1           2017                                        IT  701,012,746  328,828,874.0  372,183,872.0                                        IT 2017
2           1975                                      JAWS  482,947,378  272,257,035.0  210,690,343.0                                      JAWS 1975
3           2007                               I AM LEGEND  585,532,684  256,393,010.0  329,139,674.0                               I AM LEGEND 2007
4           1973                              THE EXORCIST  428,214,478  230,347,346.0  197,867,132.0                              THE EXORCIST 1973
5           2019                           IT: CHAPTER TWO  467,563,955  211,593,228.0  255,970,727.0                           IT: CHAPTER TWO 2019
6           2018                             A QUIET PLACE  334,876,670  188,024,361.0  146,852,309.0                             A QUIET PLACE 2018
7           2017                                   GET OUT  252,297,405  175,837,935.0   76,459,470.0                                   GET OUT 2017
8           2001                                  HANNIBAL  350,100,280  165,092,266.0  185,008,014.0                                  HANNIBAL 2001
9           2021                    A QUIET PLACE: PART II  296,650,356  160,215,764.0  136,434,592.0                    A QUIET PLACE: PART II 2021
10          2018                                 HALLOWEEN  255,416,089  159,366,015.0   96,050,074.0                                 HALLOWEEN 2018
11          1999                   THE BLAIR WITCH PROJECT  248,300,000  140,539,099.0  107,760,901.0                   THE BLAIR WITCH PROJECT 1999
12          2017                                     SPLIT  278,754,594  138,141,585.0  140,613,009.0                                     SPLIT 2017
13          2013                             THE CONJURING  317,740,900  137,400,141.0  180,340,759.0                             THE CONJURING 2013
14          1991                  THE SILENCE OF THE LAMBS  275,726,716  130,726,716.0  145,000,000.0                  THE SILENCE OF THE LAMBS 1991
15          2002                                  THE RING  248,218,486  129,094,024.0  119,124,462.0                                  THE RING 2002
16          2022                                      NOPE  171,416,477  123,277,080.0   48,139,397.0                                      NOPE 2022
17          2018                                   THE NUN  363,391,647  117,450,119.0  245,941,528.0                                   THE NUN 2018
18          2004                                THE GRUDGE  187,281,115  110,359,362.0   76,921,753.0                                THE GRUDGE 2004
19          2009                       PARANORMAL ACTIVITY  194,183,034  107,918,810.0   86,264,224.0                       PARANORMAL ACTIVITY 2009
20          2022                                     SMILE  215,978,513  105,778,513.0  110,200,000.0                                     SMILE 2022
21          1994                INTERVIEW WITH THE VAMPIRE  223,564,608  105,264,608.0  118,300,000.0                INTERVIEW WITH THE VAMPIRE 1994
22          2011                     PARANORMAL ACTIVITY 3  207,039,844  104,028,807.0  103,011,037.0                     PARANORMAL ACTIVITY 3 2011
23          1996                                    SCREAM  173,046,640  103,046,663.0   69,999,977.0                                    SCREAM 1996
24          1978                                    JAWS 2  208,900,376  102,922,376.0  105,978,000.0                                    JAWS 2 1978
25          2016  THE CONJURING 2: THE ENFIELD POLTERGEIST  310,196,332  102,470,008.0  207,726,324.0  THE CONJURING 2: THE ENFIELD POLTERGEIST 2016
26          2017                       ANNABELLE: CREATION  305,384,865  102,092,201.0  203,292,664.0                       ANNABE

In [12]:
tdbo = pd.read_excel("../data/thenumbers.xlsx", sheet_name=('Top 100 Domestic Box Office'))
tdbo.index = np.arange(1, len(tdbo) + 1)
tdbo = tdbo.rename(columns={'Movie' : 'Title', 'Released' : 'Release Year', 'Infl. Adj. Dom.' : 'Domestic - Adjusted for Inflation'})
tdbo['Domestic']=tdbo['Domestic'].apply('{:,}'.format)
tdbo['Domestic - Adjusted for Inflation']=tdbo['Domestic - Adjusted for Inflation'].apply('{:,}'.format)
tdbo['Title'] = tdbo['Title'].str.upper()

In [123]:
tdbo['Title'] = tdbo['Title'].fillna(value=1408)

In [124]:
tdbo

Release Year                                              Title     Domestic Domestic - Adjusted for Inflation
1            2017                                                 IT  328,828,874                       334,783,459
2            1973                                       THE EXORCIST  230,347,346                     1,063,304,892
3            2019                                     IT CHAPTER TWO  211,593,228                       211,824,221
4            2018                                      A QUIET PLACE  188,024,361                       189,262,720
5            2017                                            GET OUT  175,837,935                       179,752,944
6            2020                              A QUIET PLACE PART II  160,215,764                       160,215,764
7            2018                                          HALLOWEEN  159,366,015                       160,409,948
8            1999                            THE BLAIR WITCH PROJECT  140,539,099                       253,689,664
9            2016                                              SPLIT  138,141,585                       141,221,659
10           2013                                      THE CONJURING  137,400,141                       154,976,540
11           2002                                           THE RING  129,094,024                       203,643,481
12           2022                                               NOPE  123,277,080                       123,277,080
13           2018                                            THE NUN  117,450,119                       118,223,666
14           2004                                         THE GRUDGE  110,359,362                       162,962,207
15           2007                                PARANORMAL ACTIVITY  107,918,810                       131,944,856
16           2022                                              SMILE  105,778,513                       105,778,513
17           1994  INTERVIEW WITH THE VAMPIRE: THE VAMPIRE CHRONI...  105,264,608                       235,963,201
18           2011                              PARANORMAL ACTIVITY 3  104,028,807                       120,295,508
19           1996                                             SCREAM  103,046,663                       208,875,525
20           2016                                    THE CONJURING 2  102,470,008                       108,630,057
21           2017                                ANNABELLE: CREATION  102,092,201                       104,368,502
22           1997                                           SCREAM 2  101,363,301                       201,830,911
23           1999                                      SLEEPY HOLLOW  101,068,340                       181,585,991
24           2001                                         THE OTHERS   96,522,687                       156,380,393
25           2021                                    HALLOWEEN KILLS   92,002,155                        92,002,155
26           1999                                       THE HAUNTING   91,188,905                       164,606,736
27           2021                                    THE BLACK PHONE   89,887,230                        89,887,230
28           2016                                      DON'T BREATHE   89,217,875                        94,581,260
29           2000                                           SCREAM 3   89,138,076                       151,650,489
30           2005                                             SAW II   87,025,093                       124,495,771
31           1979                              THE AMITYVILLE HORROR   86,432,520                       315,771,389
32           2010                              PARANORMAL ACTIVITY 2   84,752,907                        98,501,884
33           2014                                          ANNABELLE   84,273,813                        94,588,843
34           2013                               INSIDIOUS: CH

In [15]:
top100comb = pd.merge(tdbo, imdbdf, left_on = ['Title', 'Release Year'], right_on = ['Title', 'Release Year'], how='left')
top100comb.index = np.arange(1, len(top100comb) + 1)

In [16]:
top100comb

Release Year                                              Title     Domestic Domestic - Adjusted for Inflation  Unnamed: 0 Rating  Runtime IMDB Score     Metascore
1            2017                                                 IT  328,828,874                       334,783,459        45.0      R    135.0        7.3            69
2            1973                                       THE EXORCIST  230,347,346                     1,063,304,892        71.0      R    122.0        8.1            81
3            2019                                     IT CHAPTER TWO  211,593,228                       211,824,221       128.0      R    169.0        6.5            58
4            2018                                      A QUIET PLACE  188,024,361                       189,262,720        69.0  PG-13     90.0        7.5            82
5            2017                                            GET OUT  175,837,935                       179,752,944        23.0      R    104.0        7.7            85
6            2020                              A QUIET PLACE PART II  160,215,764                       160,215,764        96.0  PG-13     97.0        7.2            71
7            2018                                          HALLOWEEN  159,366,015                       160,409,948       354.0      R    106.0        6.5            67
8            1999                            THE BLAIR WITCH PROJECT  140,539,099                       253,689,664       316.0      R     81.0        6.5            81
9            2016                                              SPLIT  138,141,585                       141,221,659        50.0  PG-13    117.0        7.3            62
10           2013                                      THE CONJURING  137,400,141                       154,976,540        73.0      R    112.0        7.5            68
11           2002                                           THE RING  129,094,024                       203,643,481       209.0  PG-13    115.0        7.1            57
12           2022                                               NOPE  123,277,080                       123,277,080         7.0      R    130.0        6.9            77
13           2018                                            THE NUN  117,450,119                       118,223,666       179.0      R     96.0        5.3            46
14           2004                                         THE GRUDGE  110,359,362                       162,962,207       580.0  PG-13     91.0        5.9            49
15           2007                                PARANORMAL ACTIVITY  107,918,810                       131,944,856       433.0      R     86.0        6.3            68
16           2022                                              SMILE  105,778,513                       105,778,513         4.0      R    115.0        6.6            68
17           1994  INTERVIEW WITH THE VAMPIRE: THE VAMPIRE CHRONI...  105,264,608                       235,963,201        60.0      R    123.0        7.5            59
18           2011                              PARANORMAL ACTIVITY 3  104,028,807                       120,295,508      1356.0      R     83.0        5.8            59
19           1996                                             SCREAM  103,046,663                       208,875,525        81.0      R    111.0        7.4            65
20           2016                                    THE CONJURING 2  102,470,008                       108,630,057       219.0      R    134.0        7.3            65
21           2017                                ANNABELLE: CREATION  102,092,201                       104,368,502       507.0      R    109.0        6.5            62
22           1997                                           SCREAM 2  101,363,301                       201,830,911       338.0      R    120.0        6.3            63
23           1999                                      SLEEPY HOLLOW  101,068,340                       181,585,991   

In [17]:
worldwidecomb = pd.merge(imdbdf, wwbo, left_on = ['Title', 'Release Year'], right_on = ['Title', 'Release Year'], how='inner')
worldwidecomb.index = np.arange(1, len(top100comb) + 1)

ValueError: Length mismatch: Expected axis has 828 elements, new values have 100 elements

In [100]:
imdbdf['Release Year'] = imdbdf['Release Year'].astype(str)

In [101]:
imdbdf.dtypes

Title             object
Release Year      object
Rating            object
Runtime           object
IMDB Score        object
Metascore         object
Title and Year    object
dtype: object

In [102]:
imdbdf['Title and Year'] = imdbdf['Title'] + " " + imdbdf['Release Year']

In [104]:
imdbdf.head()

Title Release Year Rating Runtime IMDB Score Metascore        Title and Year
1       THE MENU       2022.0      R   107.0        7.5        71       THE MENU 2022.0
2              X       2022.0      R   105.0        6.6        79              X 2022.0
3  BONES AND ALL       2022.0      R   131.0        7.3        74  BONES AND ALL 2022.0
4          SMILE       2022.0      R   115.0        6.6        68          SMILE 2022.0
5      BARBARIAN       2022.0      R   102.0        7.1        78      BARBARIAN 2022.0

In [105]:
wwbo['Release Year'] = wwbo['Release Year'].astype(str)

In [107]:
wwbo['Title and Year'] = wwbo['Title'] + " " + wwbo['Release Year']

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [108]:
wwbo.dtypes

Release Year      object
Title             object
Worldwide         object
Domestic          object
International     object
Title and Year    object
dtype: object

In [109]:
wwbo

Release Year                                 Title    Worldwide       Domestic  International                             Title and Year
1            2017                                    IT  701,012,746  328,828,874.0  372,183,872.0                                    IT 2017
2            1975                                  JAWS  482,947,378  272,257,035.0  210,690,343.0                                  JAWS 1975
3            2007                           I AM LEGEND  585,532,684  256,393,010.0  329,139,674.0                           I AM LEGEND 2007
4            1973                          THE EXORCIST  428,214,478  230,347,346.0  197,867,132.0                          THE EXORCIST 1973
5            2019                       IT: CHAPTER TWO  467,563,955  211,593,228.0  255,970,727.0                       IT: CHAPTER TWO 2019
...           ...                                   ...          ...            ...            ...                                        ...
1851         2018                          MANIAC TALES          155            nan          155.0                          MANIAC TALES 2018
1852         1933                     THE INVISIBLE MAN          132            nan          132.0                     THE INVISIBLE MAN 1933
1853         2016                  CAPTURE KILL RELEASE           72            nan           72.0                  CAPTURE KILL RELEASE 2016
1854         2021                SLUMBER PARTY MASSACRE           28            nan           28.0                SLUMBER PARTY MASSACRE 2021
1855         2019  ANTRUM: THE DEADLIEST FILM EVER MADE           26            nan           26.0  ANTRUM: THE DEADLIEST FILM EVER MADE 2019

[1855 rows x 6 columns]

In [60]:
wwbo.to_csv('wwbo.csv')

In [111]:
imdbdf.dtypes

Title             object
Release Year      object
Rating            object
Runtime           object
IMDB Score        object
Metascore         object
Title and Year    object
dtype: object

In [112]:
imdbdf['Runtime'] = imdbdf['Runtime'].astype(str)

In [113]:
imdbdf

Title Release Year     Rating Runtime      IMDB Score     Metascore                     Title and Year
1                       THE MENU       2022.0          R   107.0             7.5            71                    THE MENU 2022.0
2                              X       2022.0          R   105.0             6.6            79                           X 2022.0
3                  BONES AND ALL       2022.0          R   131.0             7.3            74               BONES AND ALL 2022.0
4                          SMILE       2022.0          R   115.0             6.6            68                       SMILE 2022.0
5                      BARBARIAN       2022.0          R   102.0             7.1            78                   BARBARIAN 2022.0
...                          ...          ...        ...     ...             ...           ...                                ...
9603       WALKING DEAD - TOMATE       2022.0  NO RATING    79.0             3.6  NO METASCORE       WALKING DEAD - TOMATE 2022.0
9604                   MANNEQUIN       2023.0  NO RATING     nan  NO IMDB RATING  NO METASCORE                   MANNEQUIN 2023.0
9605  UNDEAD OR ALIVE: A ZOMBEDY       2007.0          R    91.0             5.1  NO METASCORE  UNDEAD OR ALIVE: A ZOMBEDY 2007.0
9606               SECOND ORIGIN       2015.0  NO RATING   106.0             4.5  NO METASCORE               SECOND ORIGIN 2015.0
9607                       BETSY       2018.0  NO RATING    81.0             3.1  NO METASCORE                       BETSY 2018.0

[9607 rows x 7 columns]

In [114]:
# worldwidecomb = pd.merge(imdbdf, wwbo, left_on = ['Title', 'Release Year'], right_on = ['Title', 'Release Year'], how='inner')
# worldwidecomb.index = np.arange(1, len(top100comb) + 1)

# df2['team'] = df2['team'].apply(lambda x: difflib.get_close_matches(x, df1['team'])[0])

#wwbo['Title and Year'] = wwbo['Title and Year'].map(lambda x: difflib.get_close_matches(x, imdbdf['Title and Year'])[0])

#test = imdbdf.merge(wwbo)

#df2.index = df2.index.map(lambda x: difflib.get_close_matches(x, df1.index)[0])

for x in wwbo['Title and Year']:
    difflib.get_close_matches(x, imdbdf['Title and Year'])[0]
    print(x)
    print(difflib.get_close_matches(x, imdbdf['Title and Year'])[0])

IT 2017
IT 2017.0
JAWS 1975
JAWS 2 1978.0
I AM LEGEND 2007
BIG LEGEND 2018.0
THE EXORCIST 1973
THE EXORCIST 1973.0
IT: CHAPTER TWO 2019
IT CHAPTER TWO 2019.0
A QUIET PLACE 2018
A QUIET PLACE 2018.0
GET OUT 2017
GET OUT 2017.0
HANNIBAL 2001
CANNIBAL 2010.0
A QUIET PLACE: PART II 2021
A QUIET PLACE PART II 2020.0
HALLOWEEN 2018
HALLOWEEN 2018.0
THE BLAIR WITCH PROJECT 1999
THE BLAIR WITCH PROJECT 1999.0
SPLIT 2017
SPLIT 2016.0
THE CONJURING 2013
THE CONJURING 2013.0
THE SILENCE OF THE LAMBS 1991
THE SHIVER OF THE VAMPIRES 1971.0
THE RING 2002
THE RING 2002.0
NOPE 2022
NOPE 2022.0
THE NUN 2018
THE NUN 2018.0
THE GRUDGE 2004
THE GRUDGE 2004.0
PARANORMAL ACTIVITY 2009
PARANORMAL ACTIVITY 2007.0
SMILE 2022
SMILE 2022.0
INTERVIEW WITH THE VAMPIRE 1994
INTERVIEW WITH THE VAMPIRE: THE VAMPIRE CHRONICLES 1994.0
PARANORMAL ACTIVITY 3 2011
PARANORMAL ACTIVITY 3 2011.0
SCREAM 1996
SCREAM 1996.0
JAWS 2 1978
JAWS 2 1978.0
THE CONJURING 2: THE ENFIELD POLTERGEIST 2016
THE CONJURING: THE DEVIL MADE ME 

TypeError: 'float' object is not iterable

In [117]:
imdbdf.loc[(imdbdf['Title'] == '1408')]

Title Release Year Rating Runtime IMDB Score Metascore Title and Year
325  1408       2007.0  PG-13   104.0        6.8        64    1408 2007.0

In [129]:
wwbo.loc[(wwbo['Title'] == 1408)]

Release Year Title    Worldwide      Domestic International Title and Year
55           2007  1408  131,263,370  71,985,628.0  59,277,742.0            NaN
594          2016  1408      922,727     779,820.0     142,907.0            NaN
744          2011  1408    2,756,875      38,100.0   2,718,775.0            NaN
959          2018  1408    3,130,260           nan   3,130,260.0            NaN
1347         2020  1408      123,784           nan     123,784.0            NaN

In [127]:
tdbo.loc[(tdbo['Title'] == 1408)]

Release Year Title    Domestic Domestic - Adjusted for Inflation
49          2007  1408  71,985,628                        95,945,958

In [120]:
tdbo.head(100)

Release Year                                              Title     Domestic Domestic - Adjusted for Inflation
1            2017                                                 IT  328,828,874                       334,783,459
2            1973                                       THE EXORCIST  230,347,346                     1,063,304,892
3            2019                                     IT CHAPTER TWO  211,593,228                       211,824,221
4            2018                                      A QUIET PLACE  188,024,361                       189,262,720
5            2017                                            GET OUT  175,837,935                       179,752,944
6            2020                              A QUIET PLACE PART II  160,215,764                       160,215,764
7            2018                                          HALLOWEEN  159,366,015                       160,409,948
8            1999                            THE BLAIR WITCH PROJECT  140,539,099                       253,689,664
9            2016                                              SPLIT  138,141,585                       141,221,659
10           2013                                      THE CONJURING  137,400,141                       154,976,540
11           2002                                           THE RING  129,094,024                       203,643,481
12           2022                                               NOPE  123,277,080                       123,277,080
13           2018                                            THE NUN  117,450,119                       118,223,666
14           2004                                         THE GRUDGE  110,359,362                       162,962,207
15           2007                                PARANORMAL ACTIVITY  107,918,810                       131,944,856
16           2022                                              SMILE  105,778,513                       105,778,513
17           1994  INTERVIEW WITH THE VAMPIRE: THE VAMPIRE CHRONI...  105,264,608                       235,963,201
18           2011                              PARANORMAL ACTIVITY 3  104,028,807                       120,295,508
19           1996                                             SCREAM  103,046,663                       208,875,525
20           2016                                    THE CONJURING 2  102,470,008                       108,630,057
21           2017                                ANNABELLE: CREATION  102,092,201                       104,368,502
22           1997                                           SCREAM 2  101,363,301                       201,830,911
23           1999                                      SLEEPY HOLLOW  101,068,340                       181,585,991
24           2001                                         THE OTHERS   96,522,687                       156,380,393
25           2021                                    HALLOWEEN KILLS   92,002,155                        92,002,155
26           1999                                       THE HAUNTING   91,188,905                       164,606,736
27           2021                                    THE BLACK PHONE   89,887,230                        89,887,230
28           2016                                      DON'T BREATHE   89,217,875                        94,581,260
29           2000                                           SCREAM 3   89,138,076                       151,650,489
30           2005                                             SAW II   87,025,093                       124,495,771
31           1979                              THE AMITYVILLE HORROR   86,432,520                       315,771,389
32           2010                              PARANORMAL ACTIVITY 2   84,752,907                        98,501,884
33           2014                                          ANNABELLE   84,273,813                        94,588,843
34           2013                               INSIDIOUS: CH

In [ ]:
##1408 is returning too many times in wwbo because of NAN replacement. convert the others to string instead of floats and try fuzzy matching again. 